In [97]:
import pandas as pd 
import pyodbc
import sqlalchemy
import datetime as dt 
import numpy as np

#Se definen las conexiones

def get_engine():
    connection_string = "mssql+pyodbc://{0}@{2}:{1}@{2}.database.windows.net:{4}/{3}?driver=ODBC+Driver+17+for+SQL+Server".format(
                        'ko_ice_reader',
                        'XXXXX',
                        'koice',
                        'ko_ice',
                        1433)
    res = sqlalchemy.engine.create_engine(connection_string, connect_args={'autocommit': True})
    return res
    
def get_query(query):
    engine = get_engine()
    res = pd.read_sql(query, con=engine)
    return res

In [98]:
query = """ 

select distinct tt.LocationId, tt.user_id, tt.date_id, tt.is_reported, YEAR(srl.VisitStartDateTime) as year, MONTH(srl.VisitStartDateTime) as month, DAY(VisitStartDateTime) as day, sl.Question, sl.Response, srl.* from dbo.SceneReportList srl 
join dbo.SurveyList sl on sl.SessionUid = srl.SessionUId
join dbo.a_tt_10_CMP_r_traditional_data_01 tt on tt.SessionUid = srl.SessionUid
where YEAR(srl.VisitStartDateTime) = '2021' and srl.LocationClusterName = 'Traditional' and srl.UserRole = 'Auditor' and sl.Question = '¿La tienda tiene libre acceso a Clientes?' 
 
"""

In [99]:
df = pd.DataFrame(get_query(query))
#df = df.sort_values(by="VisitStartDateTime",ascending=False)
df

,LocationId,user_id,date_id,is_reported,year,month,day,Question,Response,AssetPurityId,...,SurveyCancelCallReason,IsQAEnvironment,SalesOfficeCode,SalesOfficeName,IdVisita,SalesGroupCode,SalesGroupName,TradeGroupCode,TradeGroupName,ParentSceneUid
0,5928469,45800,20210111,1,2021,1,11,¿La tienda tiene libre acceso a Clientes?,Yes,2353297,...,,None,Silao,Silao,None,MR0042,Silao,ABARROTE,ABARROTE,27f2512e-4b50-4a18-8e84-ad2dad857541
1,5928469,45800,20210111,1,2021,1,11,¿La tienda tiene libre acceso a Clientes?,Yes,2353570,...,,None,Silao,Silao,None,MR0042,Silao,ABARROTE,ABARROTE,
2,5928469,45800,20210414,1,2021,4,14,¿La tienda tiene libre acceso a Clientes?,Yes,3825953,...,,None,Silao,Silao,600600624leon3@storecheck.com2021-04-14,MR0042,Silao,ABARROTE,ABARROTE,
3,5936352,45800,20210210,1,2021,2,10,¿La tienda tiene libre acceso a Clientes?,No,2764800,...,,None,Silao,Silao,600600143leon3@storecheck.com2021-02-10,MR0042,Silao,MISCELÁNEA,MISCELÁNEA,
4,5936352,45800,20210507,1,2021,5,7,¿La tienda tiene libre acceso a Clientes?,No,4313077,...,,None,Silao,Silao,600600143leon3@storecheck.com2021-05-07,MR0042,Silao,MISCELÁNEA,MISCELÁNEA,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
749870,9812296,45881,20210519,1,2021,5,19,¿La tienda tiene libre acceso a Clientes?,Yes,4581285,...,,None,Tepic,Tepic,1202-1095349puertovallarta@storecheck.com2021-...,1202,Puerto Vallarta,ABARROTES,ABARROTES,
749871,9812332,45881,20210518,1,2021,5,18,¿La tienda tiene libre acceso a Clientes?,Yes,4552471,...,,None,Tepic,Tepic,1202-1105920puertovallarta@storecheck.com2021-...,1202,Puerto Vallarta,ABARROTES,ABARROTES,79aae28d-f7e7-497e-ab0b-30ff9a2a3411
749872,9812350,45881,20210517,1,2021,5,17,¿La tienda tiene libre acceso a Clientes?,Yes,4527658,...,,None,Tepic,Tepic,1202-1109765puertovallarta@storecheck.com2021-...,1202,Puerto Vallarta,ABARROTES,ABARROTES,
749873,9812369,45881,20210519,1,2021,5,19,¿La tienda tiene libre acceso a Clientes?,Yes,4570561,...,,None,Tepic,Tepic,1202-1112343puertovallarta@storecheck.com2021-...,1202,Puerto Vallarta,ABARROTES,ABARROTES,


In [52]:
df.dtypes

LocationId                               int64
user_id                                  int64
date_id                                  int64
is_reported                              int64
year                                     int64
month                                    int64
day                                      int64
Question                                object
Response                                object
AssetPurityId                            int64
UserName                                object
OutletCode                              object
Outlet                                  object
ImageCaptureTime                datetime64[ns]
SubSceneType                            object
Scene                                   object
Status                                  object
SubStatus                               object
ClientCode                              object
SubClient                               object
State                                   object
Country      

In [54]:
df['date_format']=pd.to_datetime(df[['year','month','day']])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 727883 entries, 0 to 727882
Data columns (total 48 columns):
 #   Column                        Non-Null Count   Dtype         
---  ------                        --------------   -----         
 0   LocationId                    727883 non-null  int64         
 1   user_id                       727883 non-null  int64         
 2   date_id                       727883 non-null  int64         
 3   is_reported                   727883 non-null  int64         
 4   year                          727883 non-null  int64         
 5   month                         727883 non-null  int64         
 6   day                           727883 non-null  int64         
 7   Question                      727883 non-null  object        
 8   Response                      727883 non-null  object        
 9   AssetPurityId                 727883 non-null  int64         
 10  UserName                      727883 non-null  object        
 11  OutletCode   

In [55]:
df.duplicated(subset='SessionUid', keep='first').sum()

633542

In [56]:
df=df.drop_duplicates(subset=['SessionUid'])
df

,LocationId,user_id,date_id,is_reported,year,month,day,Question,Response,AssetPurityId,...,IsQAEnvironment,SalesOfficeCode,SalesOfficeName,IdVisita,SalesGroupCode,SalesGroupName,TradeGroupCode,TradeGroupName,ParentSceneUid,date_format
0,5928469,45800,20210111,1,2021,1,11,¿La tienda tiene libre acceso a Clientes?,Yes,2353297,...,None,Silao,Silao,None,MR0042,Silao,ABARROTE,ABARROTE,27f2512e-4b50-4a18-8e84-ad2dad857541,2021-01-11
2,5928469,45800,20210414,1,2021,4,14,¿La tienda tiene libre acceso a Clientes?,Yes,3825953,...,None,Silao,Silao,600600624leon3@storecheck.com2021-04-14,MR0042,Silao,ABARROTE,ABARROTE,,2021-04-14
3,5936352,45800,20210210,1,2021,2,10,¿La tienda tiene libre acceso a Clientes?,No,2764800,...,None,Silao,Silao,600600143leon3@storecheck.com2021-02-10,MR0042,Silao,MISCELÁNEA,MISCELÁNEA,,2021-02-10
4,5936352,45800,20210507,1,2021,5,7,¿La tienda tiene libre acceso a Clientes?,No,4313077,...,None,Silao,Silao,600600143leon3@storecheck.com2021-05-07,MR0042,Silao,MISCELÁNEA,MISCELÁNEA,,2021-05-07
5,6208798,45795,20210306,1,2021,3,6,¿La tienda tiene libre acceso a Clientes?,Yes,3138998,...,None,Yucatán,Yucatán,None,9000222-31,Bodega Poniente,MISCELÁNEA / ESTANQUILLO,MISCELÁNEA / ESTANQUILLO,,2021-03-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
725468,9741122,45794,20210512,1,2021,5,12,¿La tienda tiene libre acceso a Clientes?,Yes,4408254,...,None,Yucatán,Yucatán,409400merida2@storecheck.com2021-05-12,9000222-30,Oriente,ABARROTE,ABARROTE,,2021-05-12
725521,7364277,45887,20210302,1,2021,3,2,¿La tienda tiene libre acceso a Clientes?,Yes,3051619,...,None,Cd. Juárez,Cd. Juárez,None,90005382-Cd. Juárez,Cd. Juárez,MISCELÁNEA / ESTANQUILLO,MISCELÁNEA / ESTANQUILLO,,2021-03-02
726954,7389717,52729,20210203,1,2021,2,3,¿La tienda tiene libre acceso a Clientes?,Yes,2647456,...,None,Hermosillo,Hermosillo,None,90005382-Hermosillo,Hermosillo,MISCELÁNEA / ESTANQUILLO,MISCELÁNEA / ESTANQUILLO,,2021-02-03
727484,8596418,53212,20210306,1,2021,3,6,¿La tienda tiene libre acceso a Clientes?,No,3130721,...,None,Quintana Roo,Quintana Roo,None,9000222-11,Chetumal,MISCELÁNEA / ESTANQUILLO,MISCELÁNEA / ESTANQUILLO,,2021-03-06


In [57]:
df.dtypes

LocationId                               int64
user_id                                  int64
date_id                                  int64
is_reported                              int64
year                                     int64
month                                    int64
day                                      int64
Question                                object
Response                                object
AssetPurityId                            int64
UserName                                object
OutletCode                              object
Outlet                                  object
ImageCaptureTime                datetime64[ns]
SubSceneType                            object
Scene                                   object
Status                                  object
SubStatus                               object
ClientCode                              object
SubClient                               object
State                                   object
Country      

In [58]:
table = pd.pivot_table(df, values=['Status', index=['UserName', 'OutletCode', 'user_id', 'date_id', 'is_reported', 'date_format', 'month', 'day', 'Question', 'Response', 'SessionUid'],
                    columns=['SubSceneType'], aggfunc={'Status':len}, fill_value=0)
table=pd.DataFrame(table)
table

SubSceneType                                                                                                                                                                   Comunicación Exterior (Dinámico)  \
UserName                OutletCode user_id date_id  is_reported date_format month day Question                                  Response SessionUid                                                               
Test_OpsA               10054592   53036   20210220 0           2021-02-20  2     20  ¿La tienda tiene libre acceso a Clientes? Yes     2b846a6f-08fc-4dd4-b908-90ea5224f754                                 0   
acambaro@storecheck.com 1601006950 45806   20210119 1           2021-01-19  1     19  ¿La tienda tiene libre acceso a Clientes? Yes     a12293a3-c54d-4de5-b38f-3447375cce62                                 0   
                        1601006979 45806   20210119 1           2021-01-19  1     19  ¿La tienda tiene libre acceso a Clientes? Yes     3f5f3a80-dff7-4c99-adee-c63d40b0bc48                                 1   
                        1601006984 45806   20210119 1           2021-01-19  1     19  ¿La tienda tiene libre acceso a Clientes? Yes     777daa73-49d6-47fc-a4f5-1d2d822c5cbe                                 0   
                        1601007136 45806   20210112 1           2021-01-12  1     12  ¿La tienda tiene libre acceso a Clientes? Yes     9f03c6a7-2f89-4307-b1d7-3943511563ca                                 1   
...                                                                                                                                                                                                         ...   
zamora@storecheck.com   620161330  45818   20210125 1           2021-01-25  1     25  ¿La tienda tiene libre acceso a Clientes? Yes     693dd7cb-108f-4580-8496-625c04225808                                 0   
                                           20210219 1           2021-02-19  2     19  ¿La tienda tiene libre acceso a Clientes? Yes     225d786c-a7de-4a13-a116-9e8f62fe7925                                 0   
                                           20210319 1           2021-03-19  3     19  ¿La tienda tiene libre acceso a Clientes? Yes     35ea8575-e115-42eb-82a3-01edd0308a79                                 0   
                                           20210414 1           2021-04-14  4     14  ¿La tienda tiene libre acceso a Clientes? Yes     faf5a348-7cb3-4df8-a4bc-5e9c24f0e70d                                 0   
                                           20210513 1           2021-05-13  5     13  ¿La tienda tiene libre acceso a Clientes? Yes     a29b1b45-d4d3-4add-b890-daaaaabb42f4                                 0   

SubSceneType                                                                                                                                                                   Enfriador Coca-Cola  \
UserName                OutletCode user_id date_id  is_reported date_format month day Question                                  Response SessionUid                                                  
Test_OpsA               10054592   53036   20210220 0           2021-02-20  2     20  ¿La tienda tiene libre acceso a Clientes? Yes     2b846a6f-08fc-4dd4-b908-90ea5224f754                    0   
acambaro@storecheck.com 1601006950 45806   20210119 1           2021-01-19  1     19  ¿La tienda tiene libre acceso a Clientes? Yes     a12293a3-c54d-4de5-b38f-3447375cce62                    0   
                        1601006979 45806   20210119 1           2021-01-19  1     19  ¿La tienda tiene libre acceso a Clientes? Yes     3f5f3a80-dff7-4c99-adee-c63d40b0bc48                    0   
                        1601006984 45806   20210119 1           2021-01-19  1     19  ¿La tienda tiene libre acceso a Clientes? Yes     777daa73-49d6-47fc-a4f5-1d2d822c5cbe                    0   
                        1601007136 45806   20210112 1           2021

In [59]:
table = table.reset_index()
table

SubSceneType,UserName,OutletCode,user_id,date_id,is_reported,date_format,month,day,Question,Response,SessionUid,Comunicación Exterior (Dinámico),Enfriador Coca-Cola,Enfriador Competencia,Enfriador Detallista,Exhibiciones,Precios comunicados,Rejillas
0,Test_OpsA,10054592,53036,20210220,0,2021-02-20,2,20,¿La tienda tiene libre acceso a Clientes?,Yes,2b846a6f-08fc-4dd4-b908-90ea5224f754,0,0,1,0,0,0,0
1,acambaro@storecheck.com,1601006950,45806,20210119,1,2021-01-19,1,19,¿La tienda tiene libre acceso a Clientes?,Yes,a12293a3-c54d-4de5-b38f-3447375cce62,0,0,0,0,1,0,0
2,acambaro@storecheck.com,1601006979,45806,20210119,1,2021-01-19,1,19,¿La tienda tiene libre acceso a Clientes?,Yes,3f5f3a80-dff7-4c99-adee-c63d40b0bc48,1,0,0,0,0,0,0
3,acambaro@storecheck.com,1601006984,45806,20210119,1,2021-01-19,1,19,¿La tienda tiene libre acceso a Clientes?,Yes,777daa73-49d6-47fc-a4f5-1d2d822c5cbe,0,0,0,0,1,0,0
4,acambaro@storecheck.com,1601007136,45806,20210112,1,2021-01-12,1,12,¿La tienda tiene libre acceso a Clientes?,Yes,9f03c6a7-2f89-4307-b1d7-3943511563ca,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94336,zamora@storecheck.com,620161330,45818,20210125,1,2021-01-25,1,25,¿La tienda tiene libre acceso a Clientes?,Yes,693dd7cb-108f-4580-8496-625c04225808,0,0,0,0,0,1,0
94337,zamora@storecheck.com,620161330,45818,20210219,1,2021-02-19,2,19,¿La tienda tiene libre acceso a Clientes?,Yes,225d786c-a7de-4a13-a116-9e8f62fe7925,0,0,0,0,1,0,0
94338,zamora@storecheck.com,620161330,45818,20210319,1,2021-03-19,3,19,¿La tienda tiene libre acceso a Clientes?,Yes,35ea8575-e115-42eb-82a3-01edd0308a79,0,0,0,0,0,1,0
94339,zamora@storecheck.com,620161330,45818,20210414,1,2021-04-14,4,14,¿La tienda tiene libre acceso a Clientes?,Yes,faf5a348-7cb3-4df8-a4bc-5e9c24f0e70d,0,1,0,0,0,0,0


In [7]:
table.to_excel(r"C:\Users\Lenovo\OneDrive - Storecheck S.A. de C.V\Documentos\Ifi\ifi_mapa.xlsx", sheet_name='Sheet_name_1', index=False) 

In [60]:
table.dtypes


SubSceneType
UserName                                    object
OutletCode                                  object
user_id                                      int64
date_id                                      int64
is_reported                                  int64
date_format                         datetime64[ns]
month                                        int64
day                                          int64
Question                                    object
Response                                    object
SessionUid                                  object
Comunicación Exterior (Dinámico)             int64
Enfriador Coca-Cola                          int64
Enfriador Competencia                        int64
Enfriador Detallista                         int64
Exhibiciones                                 int64
Precios comunicados                          int64
Rejillas                                     int64
dtype: object

In [61]:
ifi=table.groupby(['UserName', 'OutletCode', 'user_id', 'date_id', 'is_reported', 'date_format', 'SessionUid', 'Question', 'Response', 'month', 'day'])['Enfriador Coca-Cola', 'Enfriador Competencia', 'Enfriador Detallista'].sum()

ifi  

<ipython-input-61-196cc7a23509>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  ifi=table.groupby(['UserName', 'OutletCode', 'user_id', 'date_id', 'is_reported', 'date_format', 'SessionUid', 'Question', 'Response', 'month', 'day'])['Enfriador Coca-Cola', 'Enfriador Competencia', 'Enfriador Detallista'].sum()


SubSceneType                                                                                                                                                                   Enfriador Coca-Cola  \
UserName                OutletCode user_id date_id  is_reported date_format SessionUid                           Question                                  Response month day                        
Test_OpsA               10054592   53036   20210220 0           2021-02-20  2b846a6f-08fc-4dd4-b908-90ea5224f754 ¿La tienda tiene libre acceso a Clientes? Yes     2     20                     0   
acambaro@storecheck.com 1601006950 45806   20210119 1           2021-01-19  a12293a3-c54d-4de5-b38f-3447375cce62 ¿La tienda tiene libre acceso a Clientes? Yes     1     19                     0   
                        1601006979 45806   20210119 1           2021-01-19  3f5f3a80-dff7-4c99-adee-c63d40b0bc48 ¿La tienda tiene libre acceso a Clientes? Yes     1     19                     0   
                        1601006984 45806   20210119 1           2021-01-19  777daa73-49d6-47fc-a4f5-1d2d822c5cbe ¿La tienda tiene libre acceso a Clientes? Yes     1     19                     0   
                        1601007136 45806   20210112 1           2021-01-12  9f03c6a7-2f89-4307-b1d7-3943511563ca ¿La tienda tiene libre acceso a Clientes? Yes     1     12                     0   
...                                                                                                                                                                                            ...   
zamora@storecheck.com   620161330  45818   20210125 1           2021-01-25  693dd7cb-108f-4580-8496-625c04225808 ¿La tienda tiene libre acceso a Clientes? Yes     1     25                     0   
                                           20210219 1           2021-02-19  225d786c-a7de-4a13-a116-9e8f62fe7925 ¿La tienda tiene libre acceso a Clientes? Yes     2     19                     0   
                                           20210319 1           2021-03-19  35ea8575-e115-42eb-82a3-01edd0308a79 ¿La tienda tiene libre acceso a Clientes? Yes     3     19                     0   
                                           20210414 1           2021-04-14  faf5a348-7cb3-4df8-a4bc-5e9c24f0e70d ¿La tienda tiene libre acceso a Clientes? Yes     4     14                     1   
                                           20210513 1           2021-05-13  a29b1b45-d4d3-4add-b890-daaaaabb42f4 ¿La tienda tiene libre acceso a Clientes? Yes     5     13                     0   

SubSceneType                                                                                                                                                                   Enfriador Competencia  \
UserName                OutletCode user_id date_id  is_reported date_format SessionUid                           Question                                  Response month day                          
Test_OpsA               10054592   53036   20210220 0           2021-02-20  2b846a6f-08fc-4dd4-b908-90ea5224f754 ¿La tienda tiene libre acceso a Clientes? Yes     2     20                       1   
acambaro@storecheck.com 1601006950 45806   20210119 1           2021-01-19  a12293a3-c54d-4de5-b38f-3447375cce62 ¿La tienda tiene libre acceso a Clientes? Yes     1     19                       0   
                        1601006979 45806   20210119 1           2021-01-19  3f5f3a80-dff7-4c99-adee-c63d40b0bc48 ¿La tienda tiene libre acceso a Clientes? Yes     1     19                       0   
                        1601006984 45806   20210119 1           2021-01-19  777daa73-49d6-47fc-a4f5-1d2d822c5cbe ¿La tienda tiene libre acceso a Clientes? Yes     1     19                       0   
                        1601007136 45806   20210112 1           2021-01-12  9f03c6a7-2f89-4307-b1d7-3943511563ca ¿La tienda tiene libre acceso a Clientes? Yes     1     12                       0   
...                      

In [62]:
table2 = ifi.reset_index()
table2

SubSceneType,UserName,OutletCode,user_id,date_id,is_reported,date_format,SessionUid,Question,Response,month,day,Enfriador Coca-Cola,Enfriador Competencia,Enfriador Detallista
0,Test_OpsA,10054592,53036,20210220,0,2021-02-20,2b846a6f-08fc-4dd4-b908-90ea5224f754,¿La tienda tiene libre acceso a Clientes?,Yes,2,20,0,1,0
1,acambaro@storecheck.com,1601006950,45806,20210119,1,2021-01-19,a12293a3-c54d-4de5-b38f-3447375cce62,¿La tienda tiene libre acceso a Clientes?,Yes,1,19,0,0,0
2,acambaro@storecheck.com,1601006979,45806,20210119,1,2021-01-19,3f5f3a80-dff7-4c99-adee-c63d40b0bc48,¿La tienda tiene libre acceso a Clientes?,Yes,1,19,0,0,0
3,acambaro@storecheck.com,1601006984,45806,20210119,1,2021-01-19,777daa73-49d6-47fc-a4f5-1d2d822c5cbe,¿La tienda tiene libre acceso a Clientes?,Yes,1,19,0,0,0
4,acambaro@storecheck.com,1601007136,45806,20210112,1,2021-01-12,9f03c6a7-2f89-4307-b1d7-3943511563ca,¿La tienda tiene libre acceso a Clientes?,Yes,1,12,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94336,zamora@storecheck.com,620161330,45818,20210125,1,2021-01-25,693dd7cb-108f-4580-8496-625c04225808,¿La tienda tiene libre acceso a Clientes?,Yes,1,25,0,0,0
94337,zamora@storecheck.com,620161330,45818,20210219,1,2021-02-19,225d786c-a7de-4a13-a116-9e8f62fe7925,¿La tienda tiene libre acceso a Clientes?,Yes,2,19,0,0,0
94338,zamora@storecheck.com,620161330,45818,20210319,1,2021-03-19,35ea8575-e115-42eb-82a3-01edd0308a79,¿La tienda tiene libre acceso a Clientes?,Yes,3,19,0,0,0
94339,zamora@storecheck.com,620161330,45818,20210414,1,2021-04-14,faf5a348-7cb3-4df8-a4bc-5e9c24f0e70d,¿La tienda tiene libre acceso a Clientes?,Yes,4,14,1,0,0


In [63]:
table2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94341 entries, 0 to 94340
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   UserName               94341 non-null  object        
 1   OutletCode             94341 non-null  object        
 2   user_id                94341 non-null  int64         
 3   date_id                94341 non-null  int64         
 4   is_reported            94341 non-null  int64         
 5   date_format            94341 non-null  datetime64[ns]
 6   SessionUid             94341 non-null  object        
 7   Question               94341 non-null  object        
 8   Response               94341 non-null  object        
 9   month                  94341 non-null  int64         
 10  day                    94341 non-null  int64         
 11  Enfriador Coca-Cola    94341 non-null  int64         
 12  Enfriador Competencia  94341 non-null  int64         
 13  E

In [64]:
table2=table2.sort_values(['OutletCode', 'date_format'])

In [65]:
table2["Coca-Cola"] = table2["Enfriador Coca-Cola"].shift(1)
table2["Competencia"] = table2["Enfriador Competencia"].shift(1)
table2["Detallista"] = table2["Enfriador Detallista"].shift(1)
table2.dtypes

SubSceneType
UserName                         object
OutletCode                       object
user_id                           int64
date_id                           int64
is_reported                       int64
date_format              datetime64[ns]
SessionUid                       object
Question                         object
Response                         object
month                             int64
day                               int64
Enfriador Coca-Cola               int64
Enfriador Competencia             int64
Enfriador Detallista              int64
Coca-Cola                       float64
Competencia                     float64
Detallista                      float64
dtype: object

In [66]:
table2 = table2.fillna(0)
table2

SubSceneType,UserName,OutletCode,user_id,date_id,is_reported,date_format,SessionUid,Question,Response,month,day,Enfriador Coca-Cola,Enfriador Competencia,Enfriador Detallista,Coca-Cola,Competencia,Detallista
21876,cdmx26@storecheck.com,100000195,47713,20210312,1,2021-03-12,78f994cc-4a41-46ab-b0bb-8b11323aac57,¿La tienda tiene libre acceso a Clientes?,Yes,3,12,0,0,0,0.0,0.0,0.0
19907,cdmx24@storecheck.com,100000208,45848,20210228,1,2021-02-28,1ef2922f-4c61-47ab-a065-e94bc750a97b,¿La tienda tiene libre acceso a Clientes?,No,2,28,0,0,0,0.0,0.0,0.0
21877,cdmx26@storecheck.com,100000208,47713,20210312,1,2021-03-12,1061c10c-0134-4c44-ac84-ba4ae39b248e,¿La tienda tiene libre acceso a Clientes?,No,3,12,0,1,0,0.0,0.0,0.0
27386,cdmx36@storecheck.com,100000208,53583,20210512,1,2021-05-12,af80c334-ad2b-473e-a5aa-a7bcbda655d4,¿La tienda tiene libre acceso a Clientes?,Yes,5,12,1,0,0,0.0,1.0,0.0
23977,cdmx2@storecheck.com,100000215,45843,20210130,1,2021-01-30,f4f3d81d-f4a3-474c-80e1-8f1d711fcb38,¿La tienda tiene libre acceso a Clientes?,Yes,1,30,1,0,0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63932,merida4@storecheck.com,99361,45796,20210122,1,2021-01-22,d9689f04-47bc-4ca1-9748-6d168b1ab902,¿La tienda tiene libre acceso a Clientes?,Yes,1,22,1,0,0,0.0,0.0,1.0
63933,merida4@storecheck.com,99361,45796,20210412,1,2021-04-12,fe842bae-8ee8-49fb-b2c1-1c05d5fe7a53,¿La tienda tiene libre acceso a Clientes?,Yes,4,12,0,0,0,1.0,0.0,0.0
62748,merida3@storecheck.com,99750,45795,20210104,1,2021-01-04,9537a728-a264-4603-92d8-64e754403dc0,¿La tienda tiene libre acceso a Clientes?,No,1,4,1,0,0,0.0,0.0,0.0
63934,merida4@storecheck.com,99750,45796,20210208,1,2021-02-08,01ddcc9b-0294-4658-b0f5-4d004035e31c,¿La tienda tiene libre acceso a Clientes?,No,2,8,1,0,0,1.0,0.0,0.0


In [67]:
table2 = table2.astype({"Coca-Cola": int, "Competencia": int, "Detallista": int})
table2

SubSceneType,UserName,OutletCode,user_id,date_id,is_reported,date_format,SessionUid,Question,Response,month,day,Enfriador Coca-Cola,Enfriador Competencia,Enfriador Detallista,Coca-Cola,Competencia,Detallista
21876,cdmx26@storecheck.com,100000195,47713,20210312,1,2021-03-12,78f994cc-4a41-46ab-b0bb-8b11323aac57,¿La tienda tiene libre acceso a Clientes?,Yes,3,12,0,0,0,0,0,0
19907,cdmx24@storecheck.com,100000208,45848,20210228,1,2021-02-28,1ef2922f-4c61-47ab-a065-e94bc750a97b,¿La tienda tiene libre acceso a Clientes?,No,2,28,0,0,0,0,0,0
21877,cdmx26@storecheck.com,100000208,47713,20210312,1,2021-03-12,1061c10c-0134-4c44-ac84-ba4ae39b248e,¿La tienda tiene libre acceso a Clientes?,No,3,12,0,1,0,0,0,0
27386,cdmx36@storecheck.com,100000208,53583,20210512,1,2021-05-12,af80c334-ad2b-473e-a5aa-a7bcbda655d4,¿La tienda tiene libre acceso a Clientes?,Yes,5,12,1,0,0,0,1,0
23977,cdmx2@storecheck.com,100000215,45843,20210130,1,2021-01-30,f4f3d81d-f4a3-474c-80e1-8f1d711fcb38,¿La tienda tiene libre acceso a Clientes?,Yes,1,30,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63932,merida4@storecheck.com,99361,45796,20210122,1,2021-01-22,d9689f04-47bc-4ca1-9748-6d168b1ab902,¿La tienda tiene libre acceso a Clientes?,Yes,1,22,1,0,0,0,0,1
63933,merida4@storecheck.com,99361,45796,20210412,1,2021-04-12,fe842bae-8ee8-49fb-b2c1-1c05d5fe7a53,¿La tienda tiene libre acceso a Clientes?,Yes,4,12,0,0,0,1,0,0
62748,merida3@storecheck.com,99750,45795,20210104,1,2021-01-04,9537a728-a264-4603-92d8-64e754403dc0,¿La tienda tiene libre acceso a Clientes?,No,1,4,1,0,0,0,0,0
63934,merida4@storecheck.com,99750,45796,20210208,1,2021-02-08,01ddcc9b-0294-4658-b0f5-4d004035e31c,¿La tienda tiene libre acceso a Clientes?,No,2,8,1,0,0,1,0,0


In [68]:
table2.dtypes

SubSceneType
UserName                         object
OutletCode                       object
user_id                           int64
date_id                           int64
is_reported                       int64
date_format              datetime64[ns]
SessionUid                       object
Question                         object
Response                         object
month                             int64
day                               int64
Enfriador Coca-Cola               int64
Enfriador Competencia             int64
Enfriador Detallista              int64
Coca-Cola                         int32
Competencia                       int32
Detallista                        int32
dtype: object

In [69]:
table2["OutletCodeChanged"] = table2["OutletCode"].shift(1, fill_value=table2["OutletCode"].head(1)) != table2["OutletCode"]
table2.dtypes

SubSceneType
UserName                         object
OutletCode                       object
user_id                           int64
date_id                           int64
is_reported                       int64
date_format              datetime64[ns]
SessionUid                       object
Question                         object
Response                         object
month                             int64
day                               int64
Enfriador Coca-Cola               int64
Enfriador Competencia             int64
Enfriador Detallista              int64
Coca-Cola                         int32
Competencia                       int32
Detallista                        int32
OutletCodeChanged                  bool
dtype: object

In [70]:
table2

SubSceneType,UserName,OutletCode,user_id,date_id,is_reported,date_format,SessionUid,Question,Response,month,day,Enfriador Coca-Cola,Enfriador Competencia,Enfriador Detallista,Coca-Cola,Competencia,Detallista,OutletCodeChanged
21876,cdmx26@storecheck.com,100000195,47713,20210312,1,2021-03-12,78f994cc-4a41-46ab-b0bb-8b11323aac57,¿La tienda tiene libre acceso a Clientes?,Yes,3,12,0,0,0,0,0,0,False
19907,cdmx24@storecheck.com,100000208,45848,20210228,1,2021-02-28,1ef2922f-4c61-47ab-a065-e94bc750a97b,¿La tienda tiene libre acceso a Clientes?,No,2,28,0,0,0,0,0,0,True
21877,cdmx26@storecheck.com,100000208,47713,20210312,1,2021-03-12,1061c10c-0134-4c44-ac84-ba4ae39b248e,¿La tienda tiene libre acceso a Clientes?,No,3,12,0,1,0,0,0,0,False
27386,cdmx36@storecheck.com,100000208,53583,20210512,1,2021-05-12,af80c334-ad2b-473e-a5aa-a7bcbda655d4,¿La tienda tiene libre acceso a Clientes?,Yes,5,12,1,0,0,0,1,0,False
23977,cdmx2@storecheck.com,100000215,45843,20210130,1,2021-01-30,f4f3d81d-f4a3-474c-80e1-8f1d711fcb38,¿La tienda tiene libre acceso a Clientes?,Yes,1,30,1,0,0,1,0,0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63932,merida4@storecheck.com,99361,45796,20210122,1,2021-01-22,d9689f04-47bc-4ca1-9748-6d168b1ab902,¿La tienda tiene libre acceso a Clientes?,Yes,1,22,1,0,0,0,0,1,True
63933,merida4@storecheck.com,99361,45796,20210412,1,2021-04-12,fe842bae-8ee8-49fb-b2c1-1c05d5fe7a53,¿La tienda tiene libre acceso a Clientes?,Yes,4,12,0,0,0,1,0,0,False
62748,merida3@storecheck.com,99750,45795,20210104,1,2021-01-04,9537a728-a264-4603-92d8-64e754403dc0,¿La tienda tiene libre acceso a Clientes?,No,1,4,1,0,0,0,0,0,True
63934,merida4@storecheck.com,99750,45796,20210208,1,2021-02-08,01ddcc9b-0294-4658-b0f5-4d004035e31c,¿La tienda tiene libre acceso a Clientes?,No,2,8,1,0,0,1,0,0,False


In [71]:
table2["OutletCodeChanged"] = table2["OutletCodeChanged"].astype(object)
table2.dtypes

SubSceneType
UserName                         object
OutletCode                       object
user_id                           int64
date_id                           int64
is_reported                       int64
date_format              datetime64[ns]
SessionUid                       object
Question                         object
Response                         object
month                             int64
day                               int64
Enfriador Coca-Cola               int64
Enfriador Competencia             int64
Enfriador Detallista              int64
Coca-Cola                         int32
Competencia                       int32
Detallista                        int32
OutletCodeChanged                object
dtype: object

In [72]:
for OutletCodeChanged in table2:
      s = table2['OutletCodeChanged'].unique()
      for i in s:
          if i == 'True':
              Flag_CC = 'change'
              
          else: 
              Flag_CC = table2["Enfriador Coca-Cola"] - table2["Coca-Cola"]

              table2['Flag_CC']=Flag_CC
              
      print(Flag_CC)     

21876    0
19907    0
21877    0
27386    1
23977    0
        ..
63932    1
63933   -1
62748    1
63934    0
63935   -1
Length: 94341, dtype: int64
21876    0
19907    0
21877    0
27386    1
23977    0
        ..
63932    1
63933   -1
62748    1
63934    0
63935   -1
Length: 94341, dtype: int64
21876    0
19907    0
21877    0
27386    1
23977    0
        ..
63932    1
63933   -1
62748    1
63934    0
63935   -1
Length: 94341, dtype: int64
21876    0
19907    0
21877    0
27386    1
23977    0
        ..
63932    1
63933   -1
62748    1
63934    0
63935   -1
Length: 94341, dtype: int64
21876    0
19907    0
21877    0
27386    1
23977    0
        ..
63932    1
63933   -1
62748    1
63934    0
63935   -1
Length: 94341, dtype: int64
21876    0
19907    0
21877    0
27386    1
23977    0
        ..
63932    1
63933   -1
62748    1
63934    0
63935   -1
Length: 94341, dtype: int64
21876    0
19907    0
21877    0
27386    1
23977    0
        ..
63932    1
63933   -1
62748    1
63934  

In [73]:
table2.dtypes

SubSceneType
UserName                         object
OutletCode                       object
user_id                           int64
date_id                           int64
is_reported                       int64
date_format              datetime64[ns]
SessionUid                       object
Question                         object
Response                         object
month                             int64
day                               int64
Enfriador Coca-Cola               int64
Enfriador Competencia             int64
Enfriador Detallista              int64
Coca-Cola                         int32
Competencia                       int32
Detallista                        int32
OutletCodeChanged                object
Flag_CC                           int64
dtype: object

In [74]:
for OutletCodeChanged in table2:
      s = table2['OutletCodeChanged'].unique()
      for i in s:
          if i == 'True':
              Flag_Comp = 'change'
              
          else: 
              Flag_Comp = table2["Enfriador Competencia"] - table2["Competencia"]

              table2['Flag_Comp']=Flag_Comp
              
      print(Flag_Comp)    

21876    0
19907    0
21877    1
27386   -1
23977    0
        ..
63932    0
63933    0
62748    0
63934    0
63935    0
Length: 94341, dtype: int64
21876    0
19907    0
21877    1
27386   -1
23977    0
        ..
63932    0
63933    0
62748    0
63934    0
63935    0
Length: 94341, dtype: int64
21876    0
19907    0
21877    1
27386   -1
23977    0
        ..
63932    0
63933    0
62748    0
63934    0
63935    0
Length: 94341, dtype: int64
21876    0
19907    0
21877    1
27386   -1
23977    0
        ..
63932    0
63933    0
62748    0
63934    0
63935    0
Length: 94341, dtype: int64
21876    0
19907    0
21877    1
27386   -1
23977    0
        ..
63932    0
63933    0
62748    0
63934    0
63935    0
Length: 94341, dtype: int64
21876    0
19907    0
21877    1
27386   -1
23977    0
        ..
63932    0
63933    0
62748    0
63934    0
63935    0
Length: 94341, dtype: int64
21876    0
19907    0
21877    1
27386   -1
23977    0
        ..
63932    0
63933    0
62748    0
63934  

In [75]:
table2.dtypes

SubSceneType
UserName                         object
OutletCode                       object
user_id                           int64
date_id                           int64
is_reported                       int64
date_format              datetime64[ns]
SessionUid                       object
Question                         object
Response                         object
month                             int64
day                               int64
Enfriador Coca-Cola               int64
Enfriador Competencia             int64
Enfriador Detallista              int64
Coca-Cola                         int32
Competencia                       int32
Detallista                        int32
OutletCodeChanged                object
Flag_CC                           int64
Flag_Comp                         int64
dtype: object

In [76]:
for OutletCodeChanged in table2:
      s = table2['OutletCodeChanged'].unique()
      for i in s:
          if i == 'True':
              Flag_Det = 'change'
              
          else: 
              Flag_Det = table2["Enfriador Detallista"] - table2["Detallista"]

              table2['Flag_Det']=Flag_Det
              
      print(Flag_Det)      

21876    0
19907    0
21877    0
27386    0
23977    0
        ..
63932   -1
63933    0
62748    0
63934    0
63935    0
Length: 94341, dtype: int64
21876    0
19907    0
21877    0
27386    0
23977    0
        ..
63932   -1
63933    0
62748    0
63934    0
63935    0
Length: 94341, dtype: int64
21876    0
19907    0
21877    0
27386    0
23977    0
        ..
63932   -1
63933    0
62748    0
63934    0
63935    0
Length: 94341, dtype: int64
21876    0
19907    0
21877    0
27386    0
23977    0
        ..
63932   -1
63933    0
62748    0
63934    0
63935    0
Length: 94341, dtype: int64
21876    0
19907    0
21877    0
27386    0
23977    0
        ..
63932   -1
63933    0
62748    0
63934    0
63935    0
Length: 94341, dtype: int64
21876    0
19907    0
21877    0
27386    0
23977    0
        ..
63932   -1
63933    0
62748    0
63934    0
63935    0
Length: 94341, dtype: int64
21876    0
19907    0
21877    0
27386    0
23977    0
        ..
63932   -1
63933    0
62748    0
63934  

In [77]:
table2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 94341 entries, 21876 to 63935
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   UserName               94341 non-null  object        
 1   OutletCode             94341 non-null  object        
 2   user_id                94341 non-null  int64         
 3   date_id                94341 non-null  int64         
 4   is_reported            94341 non-null  int64         
 5   date_format            94341 non-null  datetime64[ns]
 6   SessionUid             94341 non-null  object        
 7   Question               94341 non-null  object        
 8   Response               94341 non-null  object        
 9   month                  94341 non-null  int64         
 10  day                    94341 non-null  int64         
 11  Enfriador Coca-Cola    94341 non-null  int64         
 12  Enfriador Competencia  94341 non-null  int64         
 1

In [78]:
table_final = table2[['UserName','user_id','OutletCode','date_id', 'is_reported', 'SessionUid', 'Question', 'Response', 'month', 'day', 'OutletCodeChanged', 'Enfriador Coca-Cola', 'Coca-Cola', 'Flag_CC', 'Enfriador Competencia', 'Competencia', 'Flag_Comp', 'Enfriador Detallista', 'Detallista', 'Flag_Det']]
table_final

SubSceneType,UserName,user_id,OutletCode,date_id,is_reported,SessionUid,Question,Response,month,day,OutletCodeChanged,Enfriador Coca-Cola,Coca-Cola,Flag_CC,Enfriador Competencia,Competencia,Flag_Comp,Enfriador Detallista,Detallista,Flag_Det
21876,cdmx26@storecheck.com,47713,100000195,20210312,1,78f994cc-4a41-46ab-b0bb-8b11323aac57,¿La tienda tiene libre acceso a Clientes?,Yes,3,12,False,0,0,0,0,0,0,0,0,0
19907,cdmx24@storecheck.com,45848,100000208,20210228,1,1ef2922f-4c61-47ab-a065-e94bc750a97b,¿La tienda tiene libre acceso a Clientes?,No,2,28,True,0,0,0,0,0,0,0,0,0
21877,cdmx26@storecheck.com,47713,100000208,20210312,1,1061c10c-0134-4c44-ac84-ba4ae39b248e,¿La tienda tiene libre acceso a Clientes?,No,3,12,False,0,0,0,1,0,1,0,0,0
27386,cdmx36@storecheck.com,53583,100000208,20210512,1,af80c334-ad2b-473e-a5aa-a7bcbda655d4,¿La tienda tiene libre acceso a Clientes?,Yes,5,12,False,1,0,1,0,1,-1,0,0,0
23977,cdmx2@storecheck.com,45843,100000215,20210130,1,f4f3d81d-f4a3-474c-80e1-8f1d711fcb38,¿La tienda tiene libre acceso a Clientes?,Yes,1,30,True,1,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63932,merida4@storecheck.com,45796,99361,20210122,1,d9689f04-47bc-4ca1-9748-6d168b1ab902,¿La tienda tiene libre acceso a Clientes?,Yes,1,22,True,1,0,1,0,0,0,0,1,-1
63933,merida4@storecheck.com,45796,99361,20210412,1,fe842bae-8ee8-49fb-b2c1-1c05d5fe7a53,¿La tienda tiene libre acceso a Clientes?,Yes,4,12,False,0,1,-1,0,0,0,0,0,0
62748,merida3@storecheck.com,45795,99750,20210104,1,9537a728-a264-4603-92d8-64e754403dc0,¿La tienda tiene libre acceso a Clientes?,No,1,4,True,1,0,1,0,0,0,0,0,0
63934,merida4@storecheck.com,45796,99750,20210208,1,01ddcc9b-0294-4658-b0f5-4d004035e31c,¿La tienda tiene libre acceso a Clientes?,No,2,8,False,1,1,0,0,0,0,0,0,0


In [26]:
table_final.to_excel(r"C:\Users\Lenovo\OneDrive - Storecheck S.A. de C.V\Documentos\Ifi\table_final1.xlsx", sheet_name='Sheet_name_1', index=False)

In [79]:
table_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 94341 entries, 21876 to 63935
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   UserName               94341 non-null  object
 1   user_id                94341 non-null  int64 
 2   OutletCode             94341 non-null  object
 3   date_id                94341 non-null  int64 
 4   is_reported            94341 non-null  int64 
 5   SessionUid             94341 non-null  object
 6   Question               94341 non-null  object
 7   Response               94341 non-null  object
 8   month                  94341 non-null  int64 
 9   day                    94341 non-null  int64 
 10  OutletCodeChanged      94341 non-null  object
 11  Enfriador Coca-Cola    94341 non-null  int64 
 12  Coca-Cola              94341 non-null  int32 
 13  Flag_CC                94341 non-null  int64 
 14  Enfriador Competencia  94341 non-null  int64 
 15  Competencia    

In [80]:
query = """ 

select distinct sl.SessionUId, sl.LocationId from dbo.SessionList sl
where YEAR(sl.VisitStartDateTime) = '2021' 
 

        """

In [81]:
dff = pd.DataFrame(get_query(query))
#df = df.sort_values(by="VisitStartDateTime",ascending=False)
dff=dff.rename(columns={"SessionUId": "SessionUid"})
dff.dtypes

SessionUid    object
LocationId     int64
dtype: object

In [82]:
Ifi_historico = pd.merge(table_final, dff, on='SessionUid', how='inner')
Ifi_historico

,UserName,user_id,OutletCode,date_id,is_reported,SessionUid,Question,Response,month,day,...,Enfriador Coca-Cola,Coca-Cola,Flag_CC,Enfriador Competencia,Competencia,Flag_Comp,Enfriador Detallista,Detallista,Flag_Det,LocationId
0,cdmx26@storecheck.com,47713,100000195,20210312,1,78f994cc-4a41-46ab-b0bb-8b11323aac57,¿La tienda tiene libre acceso a Clientes?,Yes,3,12,...,0,0,0,0,0,0,0,0,0,6816944
1,cdmx24@storecheck.com,45848,100000208,20210228,1,1ef2922f-4c61-47ab-a065-e94bc750a97b,¿La tienda tiene libre acceso a Clientes?,No,2,28,...,0,0,0,0,0,0,0,0,0,6828931
2,cdmx26@storecheck.com,47713,100000208,20210312,1,1061c10c-0134-4c44-ac84-ba4ae39b248e,¿La tienda tiene libre acceso a Clientes?,No,3,12,...,0,0,0,1,0,1,0,0,0,6828931
3,cdmx36@storecheck.com,53583,100000208,20210512,1,af80c334-ad2b-473e-a5aa-a7bcbda655d4,¿La tienda tiene libre acceso a Clientes?,Yes,5,12,...,1,0,1,0,1,-1,0,0,0,6828931
4,cdmx2@storecheck.com,45843,100000215,20210130,1,f4f3d81d-f4a3-474c-80e1-8f1d711fcb38,¿La tienda tiene libre acceso a Clientes?,Yes,1,30,...,1,1,0,0,0,0,0,0,0,6812427
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94336,merida4@storecheck.com,45796,99361,20210122,1,d9689f04-47bc-4ca1-9748-6d168b1ab902,¿La tienda tiene libre acceso a Clientes?,Yes,1,22,...,1,0,1,0,0,0,0,1,-1,6294575
94337,merida4@storecheck.com,45796,99361,20210412,1,fe842bae-8ee8-49fb-b2c1-1c05d5fe7a53,¿La tienda tiene libre acceso a Clientes?,Yes,4,12,...,0,1,-1,0,0,0,0,0,0,6294575
94338,merida3@storecheck.com,45795,99750,20210104,1,9537a728-a264-4603-92d8-64e754403dc0,¿La tienda tiene libre acceso a Clientes?,No,1,4,...,1,0,1,0,0,0,0,0,0,6294577
94339,merida4@storecheck.com,45796,99750,20210208,1,01ddcc9b-0294-4658-b0f5-4d004035e31c,¿La tienda tiene libre acceso a Clientes?,No,2,8,...,1,1,0,0,0,0,0,0,0,6294577


In [83]:
Ifi_historico=Ifi_historico[['UserName', 'user_id', 'OutletCode', 'LocationId', 'date_id', 'is_reported','SessionUid', 'Question', 'Response', 'month', 'day', 'OutletCodeChanged', 'Enfriador Coca-Cola', 'Flag_CC', 'Enfriador Competencia', 'Flag_Comp', 'Enfriador Detallista', 'Flag_Det']]

In [84]:
Ifi_historico.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 94341 entries, 0 to 94340
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   UserName               94341 non-null  object
 1   user_id                94341 non-null  int64 
 2   OutletCode             94341 non-null  object
 3   LocationId             94341 non-null  int64 
 4   date_id                94341 non-null  int64 
 5   is_reported            94341 non-null  int64 
 6   SessionUid             94341 non-null  object
 7   Question               94341 non-null  object
 8   Response               94341 non-null  object
 9   month                  94341 non-null  int64 
 10  day                    94341 non-null  int64 
 11  OutletCodeChanged      94341 non-null  object
 12  Enfriador Coca-Cola    94341 non-null  int64 
 13  Flag_CC                94341 non-null  int64 
 14  Enfriador Competencia  94341 non-null  int64 
 15  Flag_Comp          

In [96]:
Ifi_historico['Response'].unique()

array(['Yes\x14', 'No\x14', 'No', 'Yes', 'Si'], dtype=object)

In [85]:
import pandas as pd 
import pyodbc
import sqlalchemy
import datetime as dt 
import numpy as np

#Se definen las conexiones

def get_engine():
    connection_string = "mssql+pyodbc://{0}@{2}:{1}@{2}.database.windows.net:{4}/{3}?driver=ODBC+Driver+17+for+SQL+Server".format(
                        'jrivas',
                        'XXXXX',
                        'koice',
                        'ko_ice',
                        1433)
    res = sqlalchemy.engine.create_engine(connection_string, connect_args={'autocommit': True})
    return res


def get_query(query):
    engine = get_engine()
    res = pd.read_sql(query, con=engine)
    return res


In [86]:
engine = get_engine()
Ifi_historico.to_sql('Ifi_Enfriadores_tot', con=engine, schema='reportes', if_exists='append', index=False)

OperationalError: (pyodbc.OperationalError) ('08S01', '[08S01] [Microsoft][ODBC Driver 17 for SQL Server]TCP Provider: Se produjo un error durante el intento de conexión ya que la parte conectada no respondió adecuadamente tras un periodo de tiempo, o bien se produjo un error en la conexión establecida ya que el host conectado no ha podido responder.\r\n (10060) (SQLExecDirectW); [08S01] [Microsoft][ODBC Driver 17 for SQL Server]Communication link failure (10060)')
[SQL: INSERT INTO reportes.[Ifi_Enfriadores_tot] ([UserName], user_id, [OutletCode], [LocationId], date_id, is_reported, [SessionUid], [Question], [Response], month, day, [OutletCodeChanged], [Enfriador Coca-Cola], [Flag_CC], [Enfriador Competencia], [Flag_Comp], [Enfriador Detallista], [Flag_Det]) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)]
[parameters: (('cdmx26@storecheck.com', 47713, '100000195', 6816944, 20210312, 1, '78f994cc-4a41-46ab-b0bb-8b11323aac57', '¿La tienda tiene libre acceso a Clientes?', 